In [36]:
import importlib

# Imports
import time
import logging
import random
import datetime
import pprint
import traceback
pp = pprint.PrettyPrinter()

import RabbitServerInfo as info
import DockerRemoteAPI as api
from ELSbeat import metricbeat
import LoadTester
import Manager

# for jupyter problems
importlib.reload(LoadTester)
importlib.reload(api)
importlib.reload(info)
importlib.reload(Manager)

from Manager import PackageInfo, ApiServer

# logging
logging.basicConfig(level=logging.FATAL)
LOGGER = logging.getLogger()

# Data Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
profiler = ApiServer(name='profiler1', ip='10.2.6.177', api_port='7999', id='hkg-profiler-1')
workers = [
    ApiServer(name='profiler1', ip='10.2.6.177', api_port='7999', id='hkg-profiler-1'), # The profiler server
    ApiServer(name='worker1', ip='10.2.6.178', api_port='7999', id='hkg-profiler-2'),
    ApiServer(name='worker2', ip='10.2.7.10', api_port='7999', id='hkg-worker-2'),
    ApiServer(name='worker3', ip='10.2.7.11', api_port='7999', id='hkg-worker-3'),
]

package_oltp = PackageInfo(image_name='YOUR_HUB_ID/oltp-automated',
                      test_url='http://10.2.6.171/oltpexp', mem_limit="512m", cpu_percent_limit=50, post_delay=300,
                      device_read_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      device_write_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      environment=['postfix=exp'],)

package_fileio = PackageInfo(image_name='YOUR_HUB_ID/fileio-automated',
                      test_url='http://10.2.6.171/fileioexp', mem_limit="512m", cpu_percent_limit=50, post_delay=60,
                      device_read_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      device_write_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      environment=['postfix=exp'],)

package_cpu = PackageInfo(image_name='YOUR_HUB_ID/cpu-automated',
                      test_url='http://10.2.6.171/linpackexp', mem_limit="512m", cpu_percent_limit=50, post_delay=30,
                      device_read_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      device_write_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      environment=['postfix=exp'],)

manager = Manager.Manager(profiler=profiler, workers=workers, elastic_server_ip='10.2.6.181', elastic_server_port='9200')

In [ ]:
warmup_time = '10s'
test_time = '1m'
accepted_stats=['avg']

user_count_sequence = [1,3,5,7,9,11,9,11,13,15,17,15,13,10]

sequence_len = len(user_count_sequence)

packages = [package_cpu, package_fileio, package_oltp, ]
package_max_latencies = [700, 600, 600]

package_selection_method='random'
# package_selection_method='binpack'
# package_selection_method='dispatch'
# package_selection_method='ml'


package_sequences = []
for package in packages:
    package_sequences.append(user_count_sequence)

# manager.clear_all_workers()
worker_package_counts = []
for worker in workers:
    worker_package_count = {}
    for package in packages:
        worker_package_count[package.image_name] = 0
    worker_package_counts.append(worker_package_count)
manager.worker_package_counts = worker_package_counts

In [ ]:
manager.prepare_workers_for_experiment(packages, package_selection_method=package_selection_method, 
                                       MAX_PACKAGE_PER_WORKER=6, NORMALIZED_LATENCY_THRESHOLD=2.5, test_time=test_time)

In [ ]:
print(datetime.datetime.now())
res = manager.perform_experiment(packages, package_sequences, package_max_latencies, sequence_len, 
                           test_time=test_time, warmup_time=warmup_time,
                           package_selection_method=package_selection_method)
print(datetime.datetime.now())

In [ ]:
test_res = pd.DataFrame(data=res)
test_res.to_csv('Data/'+'Experiment_' + package_selection_method + '_' + str(datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S')) + '.csv')

In [ ]:
def save_figures(test_res, package_name = 'YOUR_HUB_ID/cpu-automated', clear=True, save=True):
    package_short_name = package_name.split('/')[-1]
    
    plt.figure(1, clear=clear)
    plt.plot(test_res[package_name + '-avg_ms'])
    plt.title('Response Time (ms)')
    if save:
        plt.savefig("Images/ResponseTime_" + package_short_name + '_' + package_selection_method + ".png")
    
    plt.figure(2, clear=clear)
    plt.plot(test_res[package_name + '-completed'])
    plt.title('Throughput (Req/min)')
    if save:
        plt.savefig("Images/Thoughput_" + package_short_name + '_' + package_selection_method + ".png")
    
    plt.figure(3, clear=clear)
    plt.plot(test_res[package_name + '-count'])
    plt.title('Container Count (#)')
    if save:
        plt.savefig("Images/ContainerCount_" + package_short_name + '_' + package_selection_method + ".png")
    
    return test_res[package_name + '-avg_ms'], test_res[package_name + '-completed'], test_res[package_name + '-count']
    
plt.figure(1, clear=True)
plt.figure(2, clear=True)
plt.figure(3, clear=True)
package_names = []

latencies = np.zeros((sequence_len,))
throughputs = np.zeros((sequence_len,))
counts = np.zeros((sequence_len,))
for package in packages:
    l,t,c = save_figures(test_res, package.image_name, clear=False, save=False)
    package_short_name = package.image_name.split('/')[-1]
    package_names.append(package_short_name)
    latencies = np.add(latencies, np.array(l))
    throughputs = np.add(throughputs, np.array(t))
    counts = np.add(counts, np.array(c))

package_names.append('total')
package_short_name = 'all'

plt.figure(1)
plt.plot(latencies)
plt.legend(package_names)
plt.savefig("Images/ResponseTime_" + package_short_name + '_' + package_selection_method + ".png")
plt.figure(2)
plt.plot(throughputs)
plt.legend(package_names)
plt.savefig("Images/Thoughput_" + package_short_name + '_' + package_selection_method + ".png")
plt.figure(3)
plt.plot(counts)
plt.legend(package_names)
plt.savefig("Images/ContainerCount_" + package_short_name + '_' + package_selection_method + ".png")
plt.figure(4, clear=True)
plt.plot(user_count_sequence)
plt.title("# of users")
plt.savefig("Images/UserCountSequence.png")